<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Without-data-standardization" data-toc-modified-id="Without-data-standardization-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Without data standardization</a></span></li><li><span><a href="#Model-definition-(Vanilla-MLP)" data-toc-modified-id="Model-definition-(Vanilla-MLP)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model definition (Vanilla MLP)</a></span></li></ul></li><li><span><a href="#Looking-at-the-model-structure" data-toc-modified-id="Looking-at-the-model-structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Looking at the model structure</a></span><ul class="toc-item"><li><span><a href="#Model-performance" data-toc-modified-id="Model-performance-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Model performance</a></span></li></ul></li><li><span><a href="#With-data-standardization" data-toc-modified-id="With-data-standardization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>With data standardization</a></span><ul class="toc-item"><li><span><a href="#Model-performance" data-toc-modified-id="Model-performance-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Model performance</a></span></li></ul></li><li><span><a href="#MLP-with-tweaks" data-toc-modified-id="MLP-with-tweaks-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MLP with tweaks</a></span><ul class="toc-item"><li><span><a href="#Model-performance" data-toc-modified-id="Model-performance-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Model performance</a></span></li><li><span><a href="#Training-metrics-changes" data-toc-modified-id="Training-metrics-changes-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Training metrics changes</a></span></li></ul></li><li><span><a href="#Deep-MLP-with-random-grid-search" data-toc-modified-id="Deep-MLP-with-random-grid-search-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Deep MLP with random grid search</a></span><ul class="toc-item"><li><span><a href="#Defining-the-grid-space" data-toc-modified-id="Defining-the-grid-space-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Defining the grid space</a></span></li><li><span><a href="#using-the-best-model-for-evaluation" data-toc-modified-id="using-the-best-model-for-evaluation-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>using the best model for evaluation</a></span></li></ul></li></ul></div>

In [3]:
import sys

In [4]:
sys.path.append("c:\\users\\chait\\anaconda3\\lib\\site-packages")

In [5]:
import pandas as pd
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import numpy as np
from sklearn.model_selection import RandomizedSearchCV

from tensorflow.keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [6]:
def convert_categorical_for_dl(x):
    if x =="ENGAGED":return 1 
    else: return 0

## Load data

In [7]:
X_train = pd.read_csv("model_data/X_train.csv")
y_train = pd.read_csv("model_data/y_train.csv")
X_validation = pd.read_csv("model_data/X_validation.csv")
y_validation = pd.read_csv("model_data/y_validation.csv")

In [8]:

batch_size = 128
num_classes = 2
epochs = 20

# the data, split between train and test sets

x_train = X_train.values
x_validation = X_validation.values


In [9]:
print(x_train.shape[0], 'train samples')

print(x_validation.shape[0], 'test samples')

12835 train samples
5502 test samples


In [10]:
X_train.shape[0]

12835

In [11]:
y_train = pd.Series(y_train.is_user_engaged).apply(lambda x : convert_categorical_for_dl(x))

In [12]:
y_validation = pd.Series(y_validation.is_user_engaged).apply(lambda x : convert_categorical_for_dl(x))

In [13]:
# convert class vectors to binary class matrices
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_validation = tensorflow.keras.utils.to_categorical(y_validation, num_classes)

### Without data standardization

### Model definition (Vanilla MLP)

In [14]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               20992     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 284,674
Trainable params: 284,674
Non-trainable params: 0
_________________________________________________________________


## Looking at the model structure

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               20992     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 284,674
Trainable params: 284,674
Non-trainable params: 0
_________________________________________________________________


In [16]:
x_train.shape

(12835, 40)

In [17]:
y_train.shape

(12835, 2)

In [18]:
model.compile(loss=['binary_crossentropy'],
              optimizer=RMSprop(),
              metrics=['accuracy', recall])

In [19]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_validation, y_validation))
score = model.evaluate(x_validation, y_validation, verbose=0)
print('Validation Loss:', score[0])
print('Validation accuracy:', score[1])

Train on 12835 samples, validate on 5502 samples
Epoch 1/20
12835/12835 [==============================] - 4s 327us/sample - loss: 1.2267 - accuracy: 0.8644 - recall: 0.8652 - val_loss: 1.2193 - val_accuracy: 0.8753 - val_recall: 0.8753
Epoch 2/20
12835/12835 [==============================] - 1s 103us/sample - loss: 0.9691 - accuracy: 0.8778 - recall: 0.8772 - val_loss: 0.9713 - val_accuracy: 0.8566 - val_recall: 0.8566
Epoch 3/20
12835/12835 [==============================] - 1s 111us/sample - loss: 0.5997 - accuracy: 0.8811 - recall: 0.8805 - val_loss: 0.3631 - val_accuracy: 0.8844 - val_recall: 0.8844
Epoch 4/20
12835/12835 [==============================] - 2s 133us/sample - loss: 0.3780 - accuracy: 0.8889 - recall: 0.8881 - val_loss: 0.3133 - val_accuracy: 0.8811 - val_recall: 0.8811
Epoch 5/20
12835/12835 [==============================] - 2s 144us/sample - loss: 0.3289 - accuracy: 0.8918 - recall: 0.8917 - val_loss: 0.2795 - val_accuracy: 0.8951 - val_recall: 0.8951
Epoch 6/20


### Model performance

Deep learning model acheived a 90% recall on train and validation, no overfitting.
    - If we are looking at an interpretable model, we can use the decision tree model or random forest with some more hyper parameter tuning.
    - If we are looking for a model with high performance but no intepretability, the MLP model is a good choice.

## With data standardization

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
sc = StandardScaler()

In [28]:
sc.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [29]:
x_train_std = sc.transform(X_train)
x_validation_std = sc.transform(X_validation)


In [30]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               20992     
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 1026      
Total params: 284,674
Trainable params: 284,674
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss=['binary_crossentropy'],
              optimizer=RMSprop(),
              metrics=['accuracy', recall])

In [32]:
history = model.fit(x_train_std, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_validation_std, y_validation))
score = model.evaluate(x_validation_std, y_validation, verbose=0)
print('Validation Loss:', score[0])
print('Validation accuracy:', score[1])

Train on 12835 samples, validate on 5502 samples
Epoch 1/20
12835/12835 [==============================] - 3s 213us/sample - loss: 0.2963 - accuracy: 0.8707 - recall: 0.8708 - val_loss: 0.2745 - val_accuracy: 0.8768 - val_recall: 0.8768
Epoch 2/20
12835/12835 [==============================] - 1s 109us/sample - loss: 0.2579 - accuracy: 0.8891 - recall: 0.8892 - val_loss: 0.2780 - val_accuracy: 0.8686 - val_recall: 0.8686
Epoch 3/20
12835/12835 [==============================] - 1s 108us/sample - loss: 0.2479 - accuracy: 0.8908 - recall: 0.8903 - val_loss: 0.2861 - val_accuracy: 0.8579 - val_recall: 0.8579
Epoch 4/20
12835/12835 [==============================] - 1s 103us/sample - loss: 0.2426 - accuracy: 0.8912 - recall: 0.8905 - val_loss: 0.2583 - val_accuracy: 0.8810 - val_recall: 0.8810
Epoch 5/20
12835/12835 [==============================] - 1s 101us/sample - loss: 0.2391 - accuracy: 0.8954 - recall: 0.8952 - val_loss: 0.2443 - val_accuracy: 0.8922 - val_recall: 0.8922
Epoch 6/20


### Model performance

The model performance with stardardized data is similar to that of the model built without standardization.
Recall of ~90% on both train and validation data

## MLP with tweaks

In [81]:
es = EarlyStopping(monitor="val_recall", patience=30, min_delta=0.5)
filepath="model_weights/weights-improvement-{epoch:02d}-recall{val_recall:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_recall', verbose=1, save_best_only=True, mode='max')

In [82]:
from tensorflow.keras.callbacks import CSVLogger

csvl = CSVLogger(
    filename='training_logs/training.log',
    separator=',', 
    append=False)

In [83]:
callbacks = [es,checkpoint,csvl]

In [140]:
# del model

In [141]:
X_train.shape

(12835, 40)

In [142]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 512)               20992     
_________________________________________________________________
dropout_22 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_23 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 1026      
Total params: 284,674
Trainable params: 284,674
Non-trainable params: 0
_________________________________________________________________


In [143]:
epochs = 1000

In [144]:
batch_size = 64

In [145]:
model.compile(loss=['binary_crossentropy'],
              optimizer=Adam(),
              metrics=['accuracy', recall])
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_validation, y_validation),callbacks = callbacks)

Train on 12835 samples, validate on 5502 samples
Epoch 1/1000
12736/12835 [============================>.] - ETA: 0s - loss: 17.6941 - accuracy: 0.8452 - recall: 0.8449
Epoch 00001: val_recall did not improve from 0.90549
12835/12835 [==============================] - 3s 197us/sample - loss: 17.5660 - accuracy: 0.8455 - recall: 0.8452 - val_loss: 9.3523 - val_accuracy: 0.8669 - val_recall: 0.8773
Epoch 2/1000
12544/12835 [============================>.] - ETA: 0s - loss: 17.1161 - accuracy: 0.8509 - recall: 0.8522
Epoch 00002: val_recall did not improve from 0.90549
12835/12835 [==============================] - 2s 141us/sample - loss: 16.7958 - accuracy: 0.8512 - recall: 0.8524 - val_loss: 29.3576 - val_accuracy: 0.8848 - val_recall: 0.8788
Epoch 3/1000
12800/12835 [============================>.] - ETA: 0s - loss: 5.0840 - accuracy: 0.8446 - recall: 0.8397
Epoch 00003: val_recall did not improve from 0.90549
12835/12835 [==============================] - 2s 130us/sample - loss: 5.071

Epoch 24/1000
12800/12835 [============================>.] - ETA: 0s - loss: 0.3090 - accuracy: 0.9000 - recall: 0.8995
Epoch 00024: val_recall did not improve from 0.90822
12835/12835 [==============================] - 2s 129us/sample - loss: 0.3091 - accuracy: 0.8999 - recall: 0.8995 - val_loss: 8.7722 - val_accuracy: 0.8971 - val_recall: 0.8971
Epoch 25/1000
12544/12835 [============================>.] - ETA: 0s - loss: 0.2674 - accuracy: 0.8984 - recall: 0.8996
Epoch 00025: val_recall did not improve from 0.90822
12835/12835 [==============================] - 2s 132us/sample - loss: 0.2679 - accuracy: 0.8982 - recall: 0.8994 - val_loss: 8.6760 - val_accuracy: 0.8984 - val_recall: 0.8973
Epoch 26/1000
12672/12835 [============================>.] - ETA: 0s - loss: 0.2749 - accuracy: 0.8982 - recall: 0.8984
Epoch 00026: val_recall did not improve from 0.90822
12835/12835 [==============================] - 2s 131us/sample - loss: 0.2741 - accuracy: 0.8983 - recall: 0.8986 - val_loss: 7

In [146]:
score = model.evaluate(x_validation, y_validation, verbose=0)
print('Validation Loss:', score[0])
print('Validation accuracy:', score[1])
print('Validation Recall:', score[2])

Validation Loss: 8.28888676851522
Validation accuracy: 0.89921844
Validation Recall: 0.89914


In [147]:
score

[8.28888676851522, 0.89921844, 0.89914]

### Model performance

The validation and train recall are ~90% I have only done a dropout and early stopping, building a deeper netwrok may improve the recall.

### Training metrics changes

In [148]:
import pandas as pd

In [149]:
logs = pd.read_csv("training_logs/training.log")

In [150]:
logs.head()

,epoch,accuracy,loss,recall,val_accuracy,val_loss,val_recall
0,0,0.845462,17.566042,0.845229,0.866867,9.352285,0.877321
1,1,0.851227,16.795758,0.852419,0.884769,29.357563,0.878763
2,2,0.844877,5.071482,0.840201,0.860051,27.594191,0.868782
3,3,0.862213,320.561889,0.863886,0.887586,104.754415,0.895308
4,4,0.849552,2659.429828,0.855930,0.874409,159.011708,0.908219


In [151]:
import plotly
import plotly.offline as pyoff
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected = True)

In [152]:
data  = []
for i in logs.columns:
    if i != "epoch":
        data.append(go.Scatter(x=logs.epoch, y=logs[i], name = i))

layout = go.Layout(title ="Training loss and accuracy changes")        
fig = go.Figure(data = data, layout= layout)

fig.show()

## Deep MLP with random grid search

In [16]:
es = EarlyStopping(monitor="val_recall", patience=10, min_delta=0.5)
filepath="model_weights/weights-improvement-gs-{epoch:02d}-recall{val_recall:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_recall', verbose=1, save_best_only=True, mode='max')

from tensorflow.keras.callbacks import CSVLogger

csvl = CSVLogger(
    filename='training_logs/grid_search.log',
    separator=',', 
    append=False)

callbacks = [es,checkpoint,csvl]

In [33]:
def build_model(optimizer='adam', dense_dims1=512,dense_dims2 = 128, dense_dims3 = 64, 
               dropout1 = 0.2, dropout2 = 0.2, dropout3 = 0.2):
    model = Sequential()
    model.add(Dense(dense_dims1, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(dropout1))
    model.add(Dense(dense_dims2, activation='relu'))
    model.add(Dropout(dropout2))
    model.add(Dense(dense_dims3, activation='relu'))
    model.add(Dropout(dropout3))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy', recall])
    return model

keras_classifier = KerasClassifier(build_model, epochs=2)

### Defining the grid space

In [34]:
n_epochs = [1000]
dense_dims1 = [128,512]
dense_dims2 = [128,512]
dense_dims3 = [128,512]
dropout1 = np.linspace(0,0.4,2)
dropout2 = np.linspace(0,0.4,2)
dropout3 = np.linspace(0,0.4,2)

In [35]:
gs = RandomizedSearchCV(keras_classifier, {'epochs': n_epochs, 'dense_dims1':dense_dims1,
                                     'dense_dims2':dense_dims2,
                                     'dense_dims3':dense_dims3,
                                     
                                    'dropout1' :dropout1,
                                    'dropout2' :dropout2,
                                    'dropout3' :dropout3})

In [36]:
gs.param_distributions

{'epochs': [1000],
 'dense_dims1': [128, 512],
 'dense_dims2': [128, 512],
 'dense_dims3': [128, 512],
 'dropout1': array([0. , 0.4]),
 'dropout2': array([0. , 0.4]),
 'dropout3': array([0. , 0.4])}

In [37]:
%%time
gs.fit(X_train,y_train,verbose=1,
        validation_data=(x_validation, y_validation),callbacks = callbacks)

Train on 10268 samples, validate on 5502 samples
Epoch 1/1000
10112/10268 [============================>.] - ETA: 0s - loss: 0.8717 - accuracy: 0.8690 - recall: 0.8659
Epoch 00001: val_recall did not improve from 0.92422
10268/10268 [==============================] - 3s 295us/sample - loss: 0.8693 - accuracy: 0.8694 - recall: 0.8659 - val_loss: 0.6733 - val_accuracy: 0.8808 - val_recall: 0.8352
Epoch 2/1000
10016/10268 [============================>.] - ETA: 0s - loss: 0.4256 - accuracy: 0.8865 - recall: 0.8871
Epoch 00002: val_recall did not improve from 0.92422
10268/10268 [==============================] - 3s 254us/sample - loss: 0.4223 - accuracy: 0.8867 - recall: 0.8876 - val_loss: 0.2985 - val_accuracy: 0.8831 - val_recall: 0.8880
Epoch 3/1000
10016/10268 [============================>.] - ETA: 0s - loss: 0.3108 - accuracy: 0.8965 - recall: 0.8968
Epoch 00003: val_recall did not improve from 0.92422
10268/10268 [==============================] - 3s 250us/sample - loss: 0.3102 - a

2567/2567 [==============================] - 0s 108us/sample - loss: 0.2496 - accuracy: 0.9038 - recall: 0.9029


Train on 10268 samples, validate on 5502 samples
Epoch 1/1000
10016/10268 [============================>.] - ETA: 0s - loss: 0.9060 - accuracy: 0.8563 - recall: 0.8558
Epoch 00001: val_recall improved from 0.92422 to 0.93840, saving model to model_weights/weights-improvement-gs-01-recall0.94.hdf5
10268/10268 [==============================] - 3s 277us/sample - loss: 0.8994 - accuracy: 0.8563 - recall: 0.8556 - val_loss: 0.7032 - val_accuracy: 0.8625 - val_recall: 0.9384
Epoch 2/1000
10048/10268 [============================>.] - ETA: 0s - loss: 0.4381 - accuracy: 0.8807 - recall: 0.8830
Epoch 00002: val_recall did not improve from 0.93840
10268/10268 [==============================] - 3s 245us/sample - loss: 0.4372 - accuracy: 0.8808 - recall: 0.8831 - val_loss: 0.3315 - val_accuracy: 0.8919 - val_recall: 0.8971
Epoch 3/1000
10176/10268 [============================>.] - ETA: 0s - loss: 0.2947 - accuracy: 0.8946 - recall: 0.8937
Epoch 00003: val_recall did not improve from 0.93840
1026

2567/2567 [==============================] - 0s 108us/sample - loss: 0.2313 - accuracy: 0.9001 - recall: 0.9012


Train on 10268 samples, validate on 5502 samples
Epoch 1/1000
 9984/10268 [============================>.] - ETA: 0s - loss: 0.8180 - accuracy: 0.8695 - recall: 0.8717
Epoch 00001: val_recall did not improve from 0.93840
10268/10268 [==============================] - 3s 297us/sample - loss: 0.8041 - accuracy: 0.8703 - recall: 0.8730 - val_loss: 0.3899 - val_accuracy: 0.8764 - val_recall: 0.8702
Epoch 2/1000
10208/10268 [============================>.] - ETA: 0s - loss: 0.4250 - accuracy: 0.8900 - recall: 0.8920
Epoch 00002: val_recall did not improve from 0.93840
10268/10268 [==============================] - 3s 337us/sample - loss: 0.4247 - accuracy: 0.8900 - recall: 0.8921 - val_loss: 0.3135 - val_accuracy: 0.8872 - val_recall: 0.8981
Epoch 3/1000
10240/10268 [============================>.] - ETA: 0s - loss: 0.2737 - accuracy: 0.8988 - recall: 0.8978
Epoch 00003: val_recall did not improve from 0.93840
10268/10268 [==============================] - 3s 302us/sample - loss: 0.2748 - a

...

Train on 10268 samples, validate on 5502 samples
Epoch 1/1000
10144/10268 [============================>.] - ETA: 0s - loss: 0.7966 - accuracy: 0.8540 - recall: 0.8498
Epoch 00001: val_recall did not improve from 0.94130
10268/10268 [==============================] - 3s 250us/sample - loss: 0.7920 - accuracy: 0.8541 - recall: 0.8498 - val_loss: 0.4453 - val_accuracy: 0.8755 - val_recall: 0.8712
Epoch 2/1000
10208/10268 [============================>.] - ETA: 0s - loss: 0.4146 - accuracy: 0.8750 - recall: 0.8732
Epoch 00002: val_recall did not improve from 0.94130
10268/10268 [==============================] - 2s 217us/sample - loss: 0.4180 - accuracy: 0.8749 - recall: 0.8731 - val_loss: 0.4019 - val_accuracy: 0.8879 - val_recall: 0.8817
Epoch 3/1000
10208/10268 [============================>.] - ETA: 0s - loss: 0.3344 - accuracy: 0.8874 - recall: 0.8852
Epoch 00003: val_recall did not improve from 0.94130
10268/10268 [==============================] - 2s 220us/sample - loss: 0.3334 - a

2567/2567 [==============================] - 0s 97us/sample - loss: 0.2621 - accuracy: 0.9030 - recall: 0.9039


Train on 10268 samples, validate on 5502 samples
Epoch 1/1000
10176/10268 [============================>.] - ETA: 0s - loss: 0.7569 - accuracy: 0.8531 - recall: 0.8562
Epoch 00001: val_recall did not improve from 0.94130
10268/10268 [==============================] - 3s 247us/sample - loss: 0.7538 - accuracy: 0.8535 - recall: 0.8564 - val_loss: 0.4450 - val_accuracy: 0.8594 - val_recall: 0.8441
Epoch 2/1000
 9952/10268 [============================>.] - ETA: 0s - loss: 0.4433 - accuracy: 0.8717 - recall: 0.8687
Epoch 00002: val_recall did not improve from 0.94130
10268/10268 [==============================] - 2s 215us/sample - loss: 0.4444 - accuracy: 0.8720 - recall: 0.8696 - val_loss: 0.3676 - val_accuracy: 0.8513 - val_recall: 0.7727
Epoch 3/1000
10208/10268 [============================>.] - ETA: 0s - loss: 0.3404 - accuracy: 0.8844 - recall: 0.8766
Epoch 00003: val_recall did not improve from 0.94130
10268/10268 [==============================] - 2s 216us/sample - loss: 0.3406 - a

2567/2567 [==============================] - 0s 98us/sample - loss: 0.2246 - accuracy: 0.9118 - recall: 0.9097


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000002CA634CEB00>, as the constructor either does not set or modifies parameter dropout3

In [38]:
gs.best_params_

{'epochs': 1000,
 'dropout3': 0.4,
 'dropout2': 0.0,
 'dropout1': 0.0,
 'dense_dims3': 512,
 'dense_dims2': 128,
 'dense_dims1': 512}

### using the best model for evaluation

In [42]:
best_model = build_model(optimizer='adam',
     dropout3 = 0.4,
     dropout2 = 0.0,
     dropout1 = 0.0,
     dense_dims3 = 512,
     dense_dims2 = 128,
     dense_dims1 = 512)

In [43]:
best_model.fit(X_train,y_train,verbose=1,
        validation_data=(x_validation, y_validation),callbacks = callbacks, epochs = 1000)

Train on 12835 samples, validate on 5502 samples
Epoch 1/1000
12672/12835 [============================>.] - ETA: 0s - loss: 0.6020 - accuracy: 0.8718 - recall: 0.8707
Epoch 00001: val_recall did not improve from 0.94130
12835/12835 [==============================] - 3s 255us/sample - loss: 0.5974 - accuracy: 0.8723 - recall: 0.8716 - val_loss: 0.3270 - val_accuracy: 0.8866 - val_recall: 0.8867
Epoch 2/1000
12768/12835 [============================>.] - ETA: 0s - loss: 0.3389 - accuracy: 0.8892 - recall: 0.8909
Epoch 00002: val_recall did not improve from 0.94130
12835/12835 [==============================] - 3s 225us/sample - loss: 0.3390 - accuracy: 0.8893 - recall: 0.8912 - val_loss: 0.2636 - val_accuracy: 0.8930 - val_recall: 0.8924
Epoch 3/1000
12608/12835 [============================>.] - ETA: 0s - loss: 0.2851 - accuracy: 0.8974 - recall: 0.8971
Epoch 00003: val_recall did not improve from 0.94130
12835/12835 [==============================] - 3s 217us/sample - loss: 0.2853 - a

In [44]:
best_model.evaluate(x_validation, y_validation)

5502/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2617971995762156, 0.8952199, 0.89551836]

__The best model from grid search gives a 89% recall on train and evaluation, there was a model with 96% recall but I have to do some research to fix the issue with randomsearchcv and keras.__